# Exploring Ebay Car Sales Data
#### A Data Quest Guided Project

### Introduction

This project works with a sampling of 50,000 [dataset](https://www.kaggle.com/orgesleka/used-cars-database/data) of used cars from _eBay Kleianzeigen_, a part of German eBay.  The data has been decleaned by DataQuest for pedagogic purposes, so we will have to tidy it up.

Here are the data fields, per DataQuest.

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - The vehicle Type.
* `yearOfRegistration` - The year in which the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `kilometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeenOnline` - When the crawler saw this ad last online.

### Helpers

Maybe I should put these in a separate .py file?


In [192]:
def describe_and_print(series):
    print("Describing {}".format(series.name))
    print()
    print("Uniques: {}".format(series.unique().shape[0]))
    print()
    print(series.describe())
    print()
    print("Lowest Values")
    print(series.value_counts().sort_index(ascending=True).head())
    print()
    print("Highest Values")
    print(series.value_counts().sort_index(ascending=False).head())

### Setup

We begin by importing the necessary libraries and reading `autos.csv` into a dataframe.  The encoding utf-8 is unsuccessful, but Latin-1 does the job.

In [193]:
import pandas as pd
import numpy as np
import datetime as dt

autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [194]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [195]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

From the `info` method of the dataframe, we make the following observations.

1. We have null entries (comprising less than 20%) to deal with in the following columns:
    * `vehicleTyple`, `gearbox`, `model`, `fueltype` and `notRepairedDamage`.    
2. Our field names are are reasonably consistent in style, but we may opt for `lowercase_snake_case` rather than camelCase.
3.  Some of our fields are already of numeric datatype, but the following fields still (some tentatively) need formatting changes to enable analysis.

    * `dateCrawled`, `dateCreated`, `lastSeen` $\rightarrow$ datetime
    * `abtest`, `notRepairedDamage` $\rightarrow$ boolean
    * `odometer`, `nrOfPictures` $\rightarrow$ int64
    * `price` $\rightarrow$ float

4.  `registration_year` and `registration_month` should be good as is: no need to turn them into datetime objects.


### Field Name Changes

I wrote some code to change the names to snake_case, which was actually trickier than I thought it would be.  A few names will have to be further modified individually.

In [196]:
new_names = []
for name in autos.columns:
    new_name = name
    extras = 0
    for i,c in enumerate(name):
        if c.isupper():
            new_name = new_name[:i+extras] + "_" + name[i:]
            extras += 1
    new_names.append(new_name.lower())
            
new_names

['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'abtest',
 'vehicle_type',
 'year_of_registration',
 'gearbox',
 'power_p_s',
 'model',
 'odometer',
 'month_of_registration',
 'fuel_type',
 'brand',
 'not_repaired_damage',
 'date_created',
 'nr_of_pictures',
 'postal_code',
 'last_seen']

In [197]:
# changing a few more names

names = ['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'ab_test',                # changed
 'vehicle_type',
 'registration_year',      # changed
 'gearbox',
 'power_ps',               # changed
 'model',
 'odometer',
 'registration_month',     # changed
 'fuel_type',
 'brand',
 'unrepaired_damage',      # changed
 'ad_created',             # changed
 'nr_of_pictures',
 'postal_code',
 'last_seen']

In [198]:
autos.columns = names

### Data Cleaning
#### Preliminaries

To help us target our cleaning efforts, let's `describe` our data and get some `value_counts`.  

In [199]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [200]:
autos.describe(include="all") # describe even non-numeric fields

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


We note that the `seller` and `offer_type` columns are pretty much useless:  49,999 out of 50,000 entries are marked "privat" and ["Angebot"](https://www.google.com/search?q=angebot+english&oq=Angebot&aqs=chrome.2.69i57j0l5.4197j0j4&sourceid=chrome&ie=UTF-8).  These categories should probably be dropped.  `nr_of_pictures` as well, which has no nonzero values.

In [201]:
autos = autos.drop(["seller", "offer_type", "nr_of_pictures"], axis=1)

The `ab_test` and `unrepaired_damage` categories also have only two possible values, but this is to be expected, given the data that they represent (basically boolean).  Similarly, there are only two `gearbox` types on the market that I am aware of:  manual and automatic, and this is a potentially important piece of data.

We have already mentioned above the various fields that need to be converted into numerical datatypes.  We will continue with these conversions.

#### Type Conversions


In [202]:
# convert prices to integers
autos["price"] = autos["price"].str.replace("$","").str.replace(",","")
autos["price"] = autos["price"].astype(int)

In [203]:
# convert odometer to integers
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","")
autos["odometer"] = autos["odometer"].astype(int)
autos.rename({"odometer" : "odometer_km"}, axis=1, inplace=True)

#### Looking for Outliers

We wrote a description printout function to help us look for outliers.  

In [204]:
describe_and_print(autos["odometer_km"])

Describing odometer_km

Uniques: 13

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

Lowest Values
5000     967
10000    264
20000    784
30000    789
40000    819
Name: odometer_km, dtype: int64

Highest Values
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64


The `odometer_km` category looks reasonable by itself, ranging from 5,000 to 150,000 km (say 3,000 to 90,000 miles).  However, the upper tail of that category is not nearly granulated enough.  Two-thirds of the cars are listed at 150,000 km or more: but there is a huge difference between a car with 150,000 km and one with, say 300,000 km.  This granularity is either an issue with how cars are listed at _eBay Kleianzeigen_ or with how the data was collected.  Unfortunately, there isn't anything I can do to recaptured this loss of information.  Of necessity we leave the `odometer` category as it is.

The `price` category, however, has a few clear outliers.  Cars listed at ridiculously low (ie, just a couple of dollars) or high ($100 million US is almost an order of magnitude more expensive than the [world's most expensive car](https://www.google.com/search?q=most+expensive+car&oq=most+ex&aqs=chrome.1.69i57j0l5.5179j0j9&sourceid=chrome&ie=UTF-8), according to a quick google.)  

In [205]:
describe_and_print(autos["price"])

Describing price

Uniques: 2357

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

Lowest Values
0    1421
1     156
2       3
3       1
5       2
Name: price, dtype: int64

Highest Values
99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
Name: price, dtype: int64


But "World's Most Expensive" is not required for a price to have outlier status.   Below I've printed a more detailed listing of the sorted value_counts.  Given that there _are_ hyper expensive cars [bought and sold](https://www.ebay.com/motors/blog/the-most-expensive-cars-sold-on-ebay-motors-in-2017/) on eBay, I will only remove the most egregious outliers.  The article I link to cites cars sold in 2017 around the \$300,000 mark, so I will cut off the high values at $350,000.

Regarding the low end of prices, since eBay is an auction site it is actually conceivable for cheap cars to be listed at \$1 initially, so we will only remove the cars listed at \$0.

In [206]:
autos = autos[autos["price"].between(1, 350000)]

print(autos["price"].value_counts().sort_index(ascending=False).head())
print("     .","     .","     .", sep="\n")
print(autos["price"].value_counts().sort_index(ascending=False).tail())

350000    1
345000    1
299000    1
295000    1
265000    1
Name: price, dtype: int64
     .
     .
     .
8      1
5      2
3      1
2      3
1    156
Name: price, dtype: int64


Whether this is a sufficient amount of outlier culling is an open question.  

### Date Related Fields

There are 5 different time related categories:  `date_crawled`, `last_seen`, `ad_created`, `registration_month` and `registration_year`.  The first two of these a derived from the crawler that collected the data, while the other three were gathered from the website by the crawler.  The final two, as noted above, do not need to have their type altered.   The remainder will need to be converted into datetime objects.  But first, let's get an idea regarding the ranges of dates that we are working with.

In [207]:
# sample of time related category strings
time_data =autos[['date_crawled', 'ad_created', 'last_seen']]
time_data[: 5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [208]:
def explore_date_data(column):
    dates = time_data[column].str.split().str[0]
    date_counts = dates.value_counts(normalize=True, dropna=False).sort_index()
    print(column, "distribution")
    print("=========================")
    print(date_counts)
    
explore_date_data("date_crawled")

date_crawled distribution
2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64


The crawler apparently was operating between March 5 and April 7, 2016.  The distribution is mostly uniform, but with less crawling being perform on the first and last couple dates in the range.

In [209]:
explore_date_data("last_seen")

last_seen distribution
2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64


The date range for `last_seen` is the same, although the distribution appears different.   Much fewer ads were last seen at the beginning of the period, while a larger amount were last seen on the final fewer days.  This is no doubt an artifact of activity of crawling itself, rather than an indication of an increase in ads on the site.

In [210]:
explore_date_data("ad_created")

ad_created distribution
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20

The `ad_created` date stretches back much further in time, with some ads having been around since 2015.  Older ads are much less common, and a distributed in an apparently random fashion.  Ads are more likely to have been placed closer to the time at hand, with a roughly uniform distribution for the period of the crawl.   There is again an unsurprising tailing off of the distribution over the final few days of the period.

### Removing False Registration Year Data

For `registration_year`, which is numerical data, we can use the `Series.describe()` method.  We should be clear that this field represents the date of _first_ registration, which I take to be more or less equivalent to the year the car was made.

In [211]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Here we have found some outliers that need to be removed.  The mean registration year of 2004 seems reasonable enough, but the standard deviation of 88 years is obviously flawed.  The absurd high and low values (as evidenced by the min of 1000 and max of 9999) are the culprits.  

We can set a hard upper bound on restration years at 2016, that being the latest year of ad creation.  For a lower bound, let us tentatively set it at 1900.

In [212]:
autos = autos[autos["registration_year"].between(1900, 2016)]

In [213]:
autos["registration_year"].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

That looks more realistic:  a mean registration year of 2002, with a standard deviation of 7 years.  If we can assume an approximately normal distribution, we could expect about 70% of cars listed to have been registered between 1995 and 2009, with 95% between 1985 and 2016.

In [214]:
autos["registration_year"].value_counts(normalize=True).head()

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
Name: registration_year, dtype: float64

In [215]:
autos["registration_year"].value_counts(normalize=True).tail()

1938    0.000021
1948    0.000021
1927    0.000021
1931    0.000021
1952    0.000021
Name: registration_year, dtype: float64

### Evaluating Price According to Brand

In [216]:
autos["brand"].value_counts(normalize=True).head()

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
Name: brand, dtype: float64

In [217]:
autos["brand"].value_counts(normalize=True).tail()

daewoo     0.001500
trabant    0.001392
rover      0.001328
lancia     0.001071
lada       0.000578
Name: brand, dtype: float64

The above value counts show that a preponderance of the cars for sale are of German make, which is not so surprising.  In fact, these top brands make up nearly 50% of the total ads.  Many of the brands have very few instances to consider.  Daewoo, for instance has about 

$$0.0015 \cdot 50,000 \approx 75.$$

Let us restrict our attentions to those brands making up more than 2% of the total.  With this as our criteria:

In [218]:
brand_counts = autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > 0.05].index
common_brands

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')

In [245]:
top_price_by_brand = {}
mean_price_by_brand = {}

for brand in common_brands:
    selected_rows = autos[autos["brand"] == brand]
    sorted_rows = selected_rows.sort_values("price", ascending=False)
    top_price_by_brand[brand] = sorted_rows.iloc[0]["price"]
    mean_price_by_brand[brand] = sorted_rows.mean()["price"].round().astype(int)
    
mean_price_by_brand = pd.Series(mean_price_by_brand, name="mean_price")
mean_price_by_brand

audi             9337
bmw              8333
ford             3749
mercedes_benz    8628
opel             2975
volkswagen       5402
Name: mean_price, dtype: int64

Note the price gap:  Audis, BMWs and Mercedes are all expensive, nearly three times the price on average as Opels and Fords. common brands.   Volkswagens are roughly in the middle.

### Price per Brand in Relation to Mileage

We propose that average price will be inversely related to average mileage (kilometerage?).  In order to compare these columns (`price` and `odometer_km`, aggregated over `brand`), we will combine the series for each of the first two columns aggregated by brand together into a single dataframe.  We already have the first of these series, so let us now make the second. 

In [222]:
autos.columns

Index(['date_crawled', 'name', 'price', 'ab_test', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

In [246]:
mean_km_by_brand = {}

for brand in common_brands:
    selected_rows = autos[autos["brand"] == brand]
    mean_km_by_brand[brand] = selected_rows.mean()["odometer_km"].round().astype(int)
    
print(mean_km_by_brand)
mean_km_by_brand = pd.Series(mean_km_by_brand, name="mean_odometer")
brand_price_km = pd.concat([mean_price_by_brand, mean_km_by_brand], axis=1)

{'audi': 129157, 'bmw': 132573, 'mercedes_benz': 130788, 'ford': 124266, 'opel': 129310, 'volkswagen': 128707}


In [247]:
brand_price_km

,mean_price,mean_odometer
audi,9337,129157
bmw,8333,132573
ford,3749,124266
mercedes_benz,8628,130788
opel,2975,129310
volkswagen,5402,128707


It appears that the data does not support our hypothesis, so we must reject.  Average mileage and average price are not obviously related.

I believe my hypothesis erred in the following way: while _average_ price and _average_ mileage are not related, surely it must be the case that the mileage of an arbitrary car is inversely related to that car's price.  Let us see if this claim holds up better.

In [254]:
kms = autos["odometer_km"].unique()
kms

array([150000,  70000,  50000,  80000,  10000,  30000, 125000,  90000,
        20000,  60000,   5000,  40000, 100000])

In [291]:
price_per_odometer = {}

for km in kms:
    select_rows = autos[autos["odometer_km"] == km]
    sorted_rows = select_rows.mean()["price"]
    price_per_odometer[km] = sorted_rows
    

In [304]:
p_per_o = pd.DataFrame.from_dict(price_per_odometer, orient='index')
p_per_o = p_per_o.reset_index()
p_per_o.rename(columns={"index":"odometer_km", 0:"mean_price"}, inplace=True)
p_per_o = p_per_o.sort_values("odometer_km", ascending=True)
p_per_o

,odometer_km,mean_price
9,5000,8873.515924
12,10000,20550.867220
0,20000,18448.477089
6,30000,16608.836842
11,40000,15499.568381
8,50000,13812.173212
10,60000,12385.004433
1,70000,10927.182814
5,80000,9721.947636
2,90000,8465.025105


From the dataframe above, is seems clear that there is a pretty clear inverse relation between mileage and average price.  The only data point bucking the trend is the low mileage of 5,000, which corresponds to an average price that is right around that of a car with 90,000 km.  It might be worth while to look more closely at cars listed with very low mileage.   